In [1]:
using CompTime, InteractiveUtils
using Test
using MacroTools: prettify

## Исходное (рекурсивное) определение.

In [2]:
function pw(n, x::T) where {T}
    if iszero(n)
        one(T)
    elseif isodd(n)
        x * pw(n - 1, x)
    else
        t = pw(n ÷ 2, x)
        t * t
    end
end

pw (generic function with 1 method)

In [3]:
Tuple(pw(n, "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

## "Наивный" генератор

In [4]:
function pw_gen_impl1(n, T, x)
    if iszero(n)
        :($(one(T)))
    elseif isodd(n)
        :($x * $(pw_gen_impl1(n - 1, T, x)))
    else
        t = pw_gen_impl1(n ÷ 2, T, x)
        :($t * $t)
    end
end

function pw_gen_impl1(n, T)
    pw_gen_impl1(n, T, :x)
end

pw_gen_impl1 (generic function with 2 methods)

In [5]:
[pw_gen_impl1(n, Int) for n in 0:5]

6-element Vector{Any}:
 1
  :(x * 1)
  :((x * 1) * (x * 1))
  :(x * ((x * 1) * (x * 1)))
  :(((x * 1) * (x * 1)) * ((x * 1) * (x * 1)))
  :(x * (((x * 1) * (x * 1)) * ((x * 1) * (x * 1))))

In [6]:
[pw_gen_impl1(n, String) for n in 0:5]

6-element Vector{Any}:
 ""
 :(x * "")
 :((x * "") * (x * ""))
 :(x * ((x * "") * (x * "")))
 :(((x * "") * (x * "")) * ((x * "") * (x * "")))
 :(x * (((x * "") * (x * "")) * ((x * "") * (x * ""))))

Недостаток - дублирование кода. Но, результат вычисления - правильный.

In [7]:
@generated function pw_gen1(::Val{n}, x::T) where {n, T}
    pw_gen_impl1(n, T)
end

function pw_gen1(n, x)
    pw_gen1(Val(n), x)
end

pw_gen1 (generic function with 2 methods)

In [8]:
Tuple(pw_gen1(n, "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [9]:
Tuple(pw_gen1(n, 2) for n in 0:5)

(1, 2, 4, 8, 16, 32)

## Введение промежуточных переменных

In [10]:
function pw_gen_impl2(n, T)
    if iszero(n)
        :($(one(T)))
    elseif isodd(n)
        :(x * $(pw_gen_impl2(n - 1, T)))
    else
        quote
            let t = $(pw_gen_impl2(n ÷ 2, T))
                t * t
            end
        end
    end
end

pw_gen_impl2 (generic function with 1 method)

In [11]:
[pw_gen_impl2(i, Int) |> prettify for i in 0:7]

8-element Vector{Any}:
 1
  :(x * 1)
  :(let t = x * 1
      t * t
  end)
  :(x * let t = x * 1
          t * t
      end)
  :(let t = let t = x * 1
              t * t
          end
      t * t
  end)
  :(x * let t = let t = x * 1
                  t * t
              end
          t * t
      end)
  :(let t = x * let t = x * 1
                  t * t
              end
      t * t
  end)
  :(x * let t = x * let t = x * 1
                      t * t
                  end
          t * t
      end)

Проблема - распухание кода! Нужно вводить промежуточные переменные...

## Генерация присваиваний для промежуточных результатов

In [12]:
function pw_gen_impl3(es, n, T)
    r = Symbol("r_", n)
    t = Symbol("t_", n)

    if iszero(n)
        push!(es, :($r = $(one(T))))
    elseif isodd(n)
        push!(es, :($r = x * $(pw_gen_impl3(es, n - 1, T))))
    else
        push!(es, :($t = $(pw_gen_impl3(es, n ÷ 2, T))))
        push!(es, :($t * $t))
    end

    return r
end

function pw_gen_impl3(n, T)
    es = Expr[]
    r = pw_gen_impl3(es, n, T)

    quote
        $(es...)
        return $r
    end
end

pw_gen_impl3 (generic function with 2 methods)

In [13]:
[pw_gen_impl3(i, Int) |> prettify for i in 0:5]

6-element Vector{Expr}:
 quote
    r_0 = 1
    return r_0
end
 quote
    r_0 = 1
    r_1 = x * r_0
    return r_1
end
 quote
    r_0 = 1
    r_1 = x * r_0
    t_2 = r_1
    t_2 * t_2
    return r_2
end
 quote
    r_0 = 1
    r_1 = x * r_0
    t_2 = r_1
    t_2 * t_2
    r_3 = x * r_2
    return r_3
end
 quote
    r_0 = 1
    r_1 = x * r_0
    t_2 = r_1
    t_2 * t_2
    t_4 = r_2
    t_4 * t_4
    return r_4
end
 quote
    r_0 = 1
    r_1 = x * r_0
    t_2 = r_1
    t_2 * t_2
    t_4 = r_2
    t_4 * t_4
    r_5 = x * r_4
    return r_5
end

## Устранение лишних присваиваний

Если переменной присваивается константа или значение другой переменной, то эту переменную можно легко устранить.

Заводим словарь, в котором будем регистрировать (символические) значения переменных.

In [14]:
function ass!(es, d, s::Symbol, c::Int64)
    d[s] = c
end

function ass!(es, d, s::Symbol, c::String)
    d[s] = c
end

function ass!(es, d, s::Symbol, u::Symbol)
    d[s] = u
end

function ass!(es, d, s::Symbol, e)
    haskey(d, s) && return
    d[s] = s
    push!(es, :($s = $e))
end

ass! (generic function with 4 methods)

Теперь генерация присваиваний будет происходить только внутри `ass!` (в случае необходимости).

In [15]:
function pw_gen_impl4(es, d, n, T)
    r = Symbol("r_", n)

    haskey(d, r) && return d[r]

    if iszero(n)
        ass!(es, d, r, :($(one(T))))
    elseif isodd(n)
        ass!(es, d, r, :(x * $(pw_gen_impl4(es, d, n - 1, T))))
    else
        t = Symbol("t_", n)
        ass!(es, d, t, pw_gen_impl4(es, d, n ÷ 2, T))
        dt = d[t]
        ass!(es, d, r, :($dt * $dt))
    end

    return d[r]
end

function pw_gen_impl4(n, T)
    es = Expr[]
    d = Dict{Symbol,Any}()
    r = pw_gen_impl4(es, d, n, T)

    quote
        $(es...)
        return $r
    end
end

pw_gen_impl4 (generic function with 2 methods)

In [16]:
[pw_gen_impl4(i, Int) |> prettify for i in 0:5]

6-element Vector{Expr}:
 :(return 1)
 quote
    r_1 = x * 1
    return r_1
end
 quote
    r_1 = x * 1
    r_2 = r_1 * r_1
    return r_2
end
 quote
    r_1 = x * 1
    r_2 = r_1 * r_1
    r_3 = x * r_2
    return r_3
end
 quote
    r_1 = x * 1
    r_2 = r_1 * r_1
    r_4 = r_2 * r_2
    return r_4
end
 quote
    r_1 = x * 1
    r_2 = r_1 * r_1
    r_4 = r_2 * r_2
    r_5 = x * r_4
    return r_5
end

## Упрощение выражений через переписывание (`Metatheory.jl`)

Выражения вида `e * 1` и `e * ""` можно упрощать до `e`.

Для этого пустим в ход "тяжёлую артиллерию": `Metatheory.jl`.

"Стрельба из пушки по воробьям!" В данном случае, конечно, можно было бы реализовать это упрощение "вручную", но, мы всё же используем `Metatheory.jl`, для отработки методики (и применения её в более сложных случаях).

In [17]:
using Metatheory, Metatheory.Rewriters

In [18]:
opt_rules = @theory e begin
    e * 1 --> e
    e * "" --> e
end;

In [19]:
strategy = (#= Fixpoint ∘ =# Postwalk ∘ Chain)
opt_expr(e) = strategy(opt_rules)(e)

opt_expr (generic function with 1 method)

In [20]:
@test opt_expr(:(x * 1)) == :x
@test opt_expr(:((x * 1) * 1)) == :x
@test opt_expr(:(x * 2)) == :(x * 2)
@test opt_expr(:(x * "")) == :x

Test Passed

In [21]:
function ass_o!(es, d, s::Symbol, e)
    ass!(es, d, s, opt_expr(e))
end

ass_o! (generic function with 1 method)

Заменяем `ass!` на `ass_o`.

In [22]:
function pw_gen_impl5(es, d, n, T)
    r = Symbol("r_", n)

    haskey(d, r) && return d[r]

    if iszero(n)
        ass_o!(es, d, r, :($(one(T))))
    elseif isodd(n)
        ass_o!(es, d, r, :(x * $(pw_gen_impl5(es, d, n - 1, T))))
    else
        t = Symbol("t_", n)
        ass_o!(es, d, t, pw_gen_impl5(es, d, n ÷ 2, T))
        dt = d[t]
        ass_o!(es, d, r, :($dt * $dt))
    end

    return d[r]
end

function pw_gen_impl5(n, T)
    es = Expr[]
    d = Dict{Symbol,Any}()
    r = pw_gen_impl5(es, d, n, T)

    quote
        $(es...)
        return $r
    end
end

pw_gen_impl5 (generic function with 2 methods)

In [23]:
[pw_gen_impl5(i, Int) |> prettify for i in 0:5]

6-element Vector{Expr}:
 :(return 1)
 :(return x)
 quote
    r_2 = x * x
    return r_2
end
 quote
    r_2 = x * x
    r_3 = x * r_2
    return r_3
end
 quote
    r_2 = x * x
    r_4 = r_2 * r_2
    return r_4
end
 quote
    r_2 = x * x
    r_4 = r_2 * r_2
    r_5 = x * r_4
    return r_5
end

In [24]:
[pw_gen_impl5(i, String) |> prettify for i in 0:5]

6-element Vector{Expr}:
 :(return "")
 :(return x)
 quote
    r_2 = x * x
    return r_2
end
 quote
    r_2 = x * x
    r_3 = x * r_2
    return r_3
end
 quote
    r_2 = x * x
    r_4 = r_2 * r_2
    return r_4
end
 quote
    r_2 = x * x
    r_4 = r_2 * r_2
    r_5 = x * r_4
    return r_5
end

In [25]:
@generated function pw_gen5(::Val{n}, x::T) where {n, T}
    pw_gen_impl5(n, T)
end

function pw_gen5(n, x)
    pw_gen5(Val(n), x)
end

pw_gen5 (generic function with 2 methods)

In [26]:

Tuple(pw_gen5(n, "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [27]:
Tuple(pw_gen5(n, 2) for n in 0:5)

(1, 2, 4, 8, 16, 32)